In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
cache_root = "/home/tim/xai-hackathon-12-06-25/assets/huggingface"
model_id = "Qwen/Qwen2.5-7B-Instruct"   # same as above

tok = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_root, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir=cache_root,
    dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
prompt = "Write one sentence about attention mechanisms."
inputs = tok(prompt, return_tensors="pt").to(model.device)
prompt_len = inputs["input_ids"].shape[1]

with torch.no_grad():
    gen = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=False,
        return_dict_in_generate=True,
        output_scores=True,
    )

seq = gen.sequences[0]
gen_ids = seq[prompt_len:]
scores = gen.scores  # list of logits, one per generated token

logprobs = []
for i, tid in enumerate(gen_ids):
    lp = torch.log_softmax(scores[i][0], dim=-1)[tid].item()
    logprobs.append(lp)

text = tok.decode(gen_ids, skip_special_tokens=True)
print(text)
print("logprobs:", logprobs)
print("total logprob:", sum(logprobs))

 Attention mechanisms allow neural networks to focus on specific parts of input data when making predictions or decisions, improving performance in tasks such as natural language processing and image recognition.
logprobs: [-0.014956246130168438, -5.376194530981593e-05, -0.5838623046875, -0.7278666496276855, -0.0004903068183921278, -4.768360213347478e-06, -0.009762637317180634, -1.3708974620385561e-05, -0.5793320536613464, -0.0046726795844733715, -1.1920928244535389e-07, -0.04663544520735741, -0.000303818320389837, -0.06301615387201309, -0.24273931980133057, -0.018959958106279373, -0.4965890049934387, -0.01852274499833584, -0.5918033123016357, -0.03130823001265526, -0.32091835141181946, -0.15038171410560608, -0.0969526395201683, -0.4741094708442688, 0.0, -0.925885796546936, -3.814689989667386e-06, -7.676783570786938e-05, -0.00016306500765495002, -0.1457817703485489, -0.12014768272638321, -0.00011169286881340668, -0.08405431360006332]
total logprob: -5.749480303435568


In [5]:
import json

json_file = 'assets/data/MATH500.json'
dataset = json.load(open(json_file, "r"))


In [ ]:
import os

import json
import random
from tqdm import tqdm
import argparse
import pandas as pd

import torch
import transformers

from utils.parse_utils import parse_answer
from constants import *
from power_samp_utils import *





if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--save_str", action = "store", type = str, default = "results/",  dest = "save_str")
    parser.add_argument("--model", action = "store", default = "qwen", type = str, choices = ["qwen", "qwen_math", "phi", "tulu", "qwen_math_grpo", "phi_grpo"])
    parser.add_argument("--temperature", action = "store", default = 0.25, type = float, dest = "temperature")
    parser.add_argument("--dataset", action = "store", default = "MATH", type = str)
    parser.add_argument("--cot", action = "store", type = bool, default = True)
    parser.add_argument("--mcmc_steps", action = "store", type = int, default = 10)
    parser.add_argument("--device", action = "store", type = str, dest = "device", default = "cuda" if torch.cuda.is_available() else 'cpu')
    parser.add_argument("--batch_idx", action = "store", type = int, default = 0)
    parser.add_argument("--seed", action = "store", type = int, default = 0)
    args = parser.parse_args()

    random.seed(0)


    model = args.model
    device = args.device
    dataset_name = args.dataset
    cot = args.cot
    temp = args.temperature
    mcmc_steps = args.mcmc_steps

    save_str = os.path.join(args.save_str, model)
    os.makedirs(save_str, exist_ok=True)


    print(model)
    print(device)
    print(mcmc_steps)
    if model == "qwen":
        model_str = "Qwen/Qwen2.5-7B"
    elif model == "qwen_math":
        model_str = "Qwen/Qwen2.5-Math-7B"
    elif model == "qwen_math_grpo":
        model_str = "stellalisy/rethink_rlvr_reproduce-ground_truth-qwen2.5_math_7b-lr5e-7-kl0.00-step150"
    elif model == "phi":
        model_str = 'microsoft/Phi-3.5-mini-instruct'
    elif model == "tulu":
        model_str = "allenai/Llama-3.1-Tulu-3-8B-DPO"

    if dataset_name == "MATH":
        json_file = 'data/MATH500.json'
        dataset = json.load(open(json_file, "r"))



    print("dataset done")
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_str, trust_remote_code = True)
    hf_model = transformers.AutoModelForCausalLM.from_pretrained(model_str, torch_dtype="auto", device_map="auto", trust_remote_code = True).to(device)
    autoreg_sampler = AutoregressiveSampler(hf_model, tokenizer, device)

    print("loaded models")
    results = []

    start = 100*args.batch_idx
    end = 100*(args.batch_idx+1)

    for problem, data in tqdm(enumerate(dataset[start:end]), desc = "Benchmark on MATH"):
        question = data["prompt"]
        print(question)
        answer = data["answer"]

        input_text = format_prompt(question, model, tokenizer, cot)
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
        prefx = [idx.item() for idx in input_ids[0]]

        naive_temp_output = hf_model.generate(input_ids, max_new_tokens=3072, 
                                return_dict_in_generate=True, output_scores=True, do_sample = True, temperature = temp)
        
        print(tokenizer.decode(naive_temp_output[0][:, len(input_ids[0]):].squeeze().to("cpu"), skip_special_tokens=True))
        print("naive done")
        
        
        std_output = hf_model.generate(input_ids, max_new_tokens=3072, 
                                return_dict_in_generate=True, output_scores=True, do_sample = True)
        
        print(tokenizer.decode(std_output[0][:, len(input_ids[0]):].squeeze().to("cpu"), skip_special_tokens=True))
        print("std done")

        mcmc_power_samp_output, _, _, acceptance_ratio = mcmc_power_samp(autoreg_sampler, prefx, temp, mcmc_steps, max_new_tokens=3072)

        print(len(std_output))
        print(len(naive_temp_output))
        print(len(mcmc_power_samp_output))
        print(tokenizer.decode(torch.tensor([mcmc_power_samp_output], dtype=torch.long, device=device).squeeze().to("cpu"), skip_special_tokens=True))
        print("mcmc done")

        naive_generated_ids = naive_temp_output[0][:, len(input_ids[0]):].squeeze().to("cpu")
        std_generated_ids = std_output[0][:, len(input_ids[0]):].squeeze().to("cpu")
        mcmc_power_samp_ids = torch.tensor([mcmc_power_samp_output], dtype=torch.long, device=device).squeeze().to("cpu")

        naive_completion = tokenizer.decode(naive_generated_ids, skip_special_tokens=True)
        std_completion = tokenizer.decode(std_generated_ids, skip_special_tokens=True)
        mcmc_completion = tokenizer.decode(mcmc_power_samp_ids, skip_special_tokens=True)

        naive_answer = parse_answer(naive_completion)
        std_answer = parse_answer(std_completion)
        mcmc_answer = parse_answer(mcmc_completion)
        
        print(naive_answer)
        print(std_answer)
        print(mcmc_answer)
        print(question)
        print(answer)
        print(f'Acceptance: {acceptance_ratio}')


        results.append({
            "question": question,
            "correct_answer": answer,
            "naive_completion": naive_completion,
            "naive_answer": naive_answer,
            "std_completion": std_completion,
            "std_answer": std_answer,
            "mcmc_completion": mcmc_completion,
            "mcmc_answer": mcmc_answer,
        })

    
    df = pd.DataFrame(results)
    df.to_csv(os.path.join(save_str, model+"_math_base_power_samp_results_" + str(mcmc_steps) + "_" + str(temp) + "_" + str(args.batch_idx)  + "_" + str(args.seed) + ".csv"), index=False)
    












        














500